In [1]:
!pip install transformers torch
!pip install datasets
!pip install librosa

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

# Import packages needed

In [2]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model, BertTokenizer, BertModel
from torch import nn
from torch.utils.data import DataLoader
from datasets import load_dataset
import librosa


In [3]:
# Clear GPU memory
torch.cuda.empty_cache()

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

GPU is available. Using GPU: NVIDIA A100-SXM4-40GB


# Load data and define collate function

In [5]:


def load_audio(file_path, target_sr=16000):
    # Load the audio file
    audio, sr = librosa.load(file_path, sr=target_sr)
    return audio

from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    audio_inputs = []
    transcriptions = []

    for item in batch:
        audio = load_audio(item['audio']['path'])
        audio_tensor = torch.tensor(audio).float()  # Ensure data is float

        # Ensure the tensor is 1D
        if audio_tensor.dim() != 1:
            audio_tensor = audio_tensor.squeeze()  # Remove any singleton dimensions
        if audio_tensor.dim() == 0:
            audio_tensor = audio_tensor.unsqueeze(0)  # Handle rare case of a single sample

        audio_inputs.append(audio_tensor)
        transcriptions.append(item['transcription'])

    # Pad audio sequences to the same length across the batch
    audio_inputs_padded = pad_sequence(audio_inputs, batch_first=True, padding_value=0.0)

    # Process transcriptions
    #print("transcriptions collate", transcriptions)
    transcription_inputs = text_tokenizer(transcriptions, return_tensors="pt", padding=True, truncation=True)

    return {
        'audio': audio_inputs_padded,  # Now properly padded
        'transcription': transcription_inputs
    }




# Define your DataLoader

# Load dataset and split
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")
split_dataset = dataset.train_test_split(test_size=0.20)
train_dataset = split_dataset['train']
valid_dataset = split_dataset['test']

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=collate_batch)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.w

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
speech_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h")

# Load text tokenizer and text model
text_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
text_model = BertModel.from_pretrained("bert-base-uncased")




preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [7]:
class TransformationModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TransformationModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

# Update the initialization of the transformation model
transform_model = TransformationModel(1024, 768)  # Adjust input_dim to 1024
optimizer = torch.optim.Adam(transform_model.parameters(), lr=1e-4)

In [8]:
speech_model = speech_model.to(device)
text_model = text_model.to(device)
transform_model = transform_model.to(device)

In [9]:
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    total_batches = 0

    with torch.no_grad():
        for batch in dataloader:
            # Process audio
            audio_input = batch['audio']
            inputs = processor(audio_input, sampling_rate=16000, return_tensors="pt", padding=True).input_values
            if inputs.dim() == 3 and inputs.shape[0] == 1:
              inputs = inputs.squeeze(0)
            inputs = inputs.to(device)
            speech_embeddings = speech_model(input_values=inputs).last_hidden_state.mean(dim=1)

            batch['transcription'] = batch['transcription'].to(device)
            text_embeddings = text_model(**batch['transcription']).last_hidden_state[:, 0, :]
            text_embeddings = text_embeddings.to(device)
            # Compute loss
            transformed_speech_emb = model(speech_embeddings)
            loss = nn.MSELoss()(transformed_speech_emb, text_embeddings)

            total_loss += loss.item()
            total_batches += 1

            del speech_embeddings, text_embeddings, inputs, batch['transcription']
    average_loss = total_loss / total_batches
    return average_loss


In [10]:
for epoch in range(5):
    transform_model.train()
    for step, batch in enumerate(train_dataloader):
        #print("Batch audio shape before processing:", batch['audio'].shape)

        # Process the audio to match model requirements
        # Ensure the processor does not add an extra unwanted dimension
        inputs = processor(batch['audio'], sampling_rate=16000, return_tensors="pt", padding=True).input_values

        # Correcting the dimensions: Remove the first dimension if it's unnecessary
        if inputs.dim() == 3 and inputs.shape[0] == 1:
            inputs = inputs.squeeze(0)

        inputs = inputs.to(device)

        #print("Corrected inputs shape:", inputs.shape)

        try:
            speech_embeddings = speech_model(input_values=inputs).last_hidden_state.mean(dim=1)
        except RuntimeError as e:
            print("Error processing batch:", e)
            continue  # Skip this batch and continue with the next one

        batch['transcription'] = batch['transcription'].to(device)
        text_embeddings = text_model(**batch['transcription']).last_hidden_state[:, 0, :]
        #text_embeddings = text_embeddings.to(device)
        #print("text_embeddings shape:", text_embeddings.shape)

        optimizer.zero_grad()
        transformed_speech_emb = transform_model(speech_embeddings)
        #print("transformed speech embeddings shape:", transformed_speech_emb.shape)
        loss = nn.MSELoss()(transformed_speech_emb, text_embeddings)
        loss.backward()
        optimizer.step()

        del speech_embeddings, text_embeddings, inputs, batch['transcription']

        if step % 100 == 0:
            val_loss = evaluate(transform_model, valid_dataloader)
            print(f"Validation Loss after {step+1} steps: {val_loss}")

    #print(f"Epoch {epoch}: Loss {loss.item()}")


Validation Loss after 1 steps: 0.3128757058528432
Validation Loss after 101 steps: 0.08455651854736763
Validation Loss after 201 steps: 0.05252442894536152
Validation Loss after 1 steps: 0.04821713536716344
Validation Loss after 101 steps: 0.04150638539801564
Validation Loss after 201 steps: 0.039125952346805944
Validation Loss after 1 steps: 0.03724341734982373
Validation Loss after 101 steps: 0.036556091682429896
Validation Loss after 201 steps: 0.035020858506884495
Validation Loss after 1 steps: 0.035810622174227445
Validation Loss after 101 steps: 0.03471891604886766
Validation Loss after 201 steps: 0.03455928079130357
Validation Loss after 1 steps: 0.034296538615435884
Validation Loss after 101 steps: 0.034338748311264475
Validation Loss after 201 steps: 0.033727570387877916


In [ ]:
# Clear GPU memory
torch.cuda.empty_cache()

import gc

# Garbage collect to remove any other lingering objects
gc.collect()

# Verify if GPU memory is cleared
!nvidia-smi

In [ ]:


# Load audio processor and audio model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
speech_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h")






# Evaluation function
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            audio_input = batch['audio']['array']
            inputs = processor(audio_input, sampling_rate=16000, return_tensors="pt", padding=True)
            speech_embeddings = speech_model(**inputs).last_hidden_state.mean(dim=1)

            text_inputs = text_tokenizer(batch['transcription'], return_tensors="pt", padding=True, truncation=True)
            text_embeddings = text_model(**text_inputs).last_hidden_state[:, 0, :]

            transformed_speech_emb = model(speech_embeddings)
            loss = nn.MSELoss()(transformed_speech_emb, text_embeddings)
            total_loss += loss.item()
    return total_loss / len(dataloader)

for epoch in range(5):
    transform_model.train()
    for step, batch in enumerate(train_dataloader):
        audio_input = batch['audio']['array']
        # Ensure the audio input is correctly shaped
        if audio_input.dim() == 3:
            audio_input = audio_input.squeeze(1)  # Squeeze if there is an unwanted singleton dimension

        inputs = processor(audio_input, sampling_rate=16000, return_tensors="pt", padding=True)
        speech_embeddings = speech_model(**inputs).last_hidden_state.mean(dim=1)

        text_inputs = text_tokenizer(batch['transcription'], return_tensors="pt", padding=True, truncation=True)
        text_embeddings = text_model(**text_inputs).last_hidden_state[:, 0, :]

        optimizer.zero_grad()
        transformed_speech_emb = transform_model(speech_embeddings)
        loss = nn.MSELoss()(transformed_speech_emb, text_embeddings)
        loss.backward()
        optimizer.step()

        if step % 100 == 0:
            val_loss = evaluate(transform_model, valid_dataloader)
            print(f"Validation Loss after {step+1} steps: {val_loss}")

    print(f"Epoch {epoch}: Loss {loss.item()}")

In [ ]:
# Print the first batch to see the data structure
first_batch = next(iter(train_dataloader))
print(first_batch.keys())  # Check top-level keys
#print(first_batch['audio'].keys())  # Check audio data structure
#print(first_batch['text'].keys())  # Replace 'text' with the correct key if different

In [ ]:
dataset[0]